# Tests of the main workhorse functions

In this notebook we present a proof-of-work for the main workhorse functions
used for Multiscale Semiwalk Balance(MSB) calculations. These are functions 
used for calculating arbitrary powers and truncated matrix exponentials 
as well as their traces and diagonals. Other high-level methods used for
calculating balance measures, profiles and contributions are just simple 
wrappers around the workhorse functions.

We test our efficient implementations based on eigendecompositions against
naive and inefficient but very straightforward implementations defined below.
We use a real-world network of Sampson's monks (at time $t = 4$).
As the results show, there is a perfect match between the two implementations
which means that our implementation is correct.

All tests work based on semiwalks and semiadjacency matrices, so the naive
functions automatically handle the case of $k=2$ when the ordinary adjacency
matrix is used.

Note that for the purpose of testing we use the full spectrum (all eigenvalues)
so the results are exact and therefore match the naive implementation.
However, in practice it is often more convenient to use approximations
based only on leading eigenvalues from the both ends of the spectrum.
We study accuracy of such approximations in a separate notebook.

In [1]:
from pathlib import Path
from math import factorial
import igraph as ig
import numpy as np
from scipy.sparse import csr_matrix, csr_array, identity
from scipy.sparse.linalg import expm as _expm
from msb import Balance

# Globals
ROOT = Path(".").absolute().parent
DATA = ROOT/"data"/"sampson"


In [2]:
## NAIVE IMPLEMENTATIONS FOR TESTING
def tr(X):
    return X.diagonal().sum()


## MATRIX FUNCTIONS USED IN STRONG BALANCE COMPUTATIONS
def mpow(X, k=1, force_semi=False):
    if k > 2 or force_semi:
        # Semiadjacency matrix is used only for lengths k > 2
        # unless forced
        X = (X + X.T) / 2
    if k == 0:
        return identity(X.shape[0], dtype=X.dtype)
    P = X
    for _ in range(1, k):
        P = P@X
    return P

def texp(X, k0=0, k1=30):
    Y = X.copy()
    Y.data[:] = 0
    Y.eliminate_zeros()
    for k in range(k0, k1+1):
        Y += mpow(X, k) / factorial(k)
    return Y

## MATRIX FUNCTIONS USED IN WEAK BALANCE COMPUTATIONS
def PNP_pow(N, P, k):
    Q = csr_matrix(N.shape, dtype=N.dtype)
    if k == 0:
        return identity(Q.shape[0], dtype=Q.dtype)
    for l in range(1, k+1):
        Q += mpow(P, l-1, force_semi=True)@N@mpow(P, k-l, force_semi=True)
    return Q

def PNP_texp(N, P, k0, k1, beta=1):
    Q = csr_matrix(N.shape, dtype=N.dtype)
    for k in range(k0, k1+1):
        Q += PNP_pow(N, P, k) * beta**k / factorial(k)
    return Q


In [3]:
## Number of powers to check
K = 30
## Inverse temperature values
BETA = [1/3, 1/2, 1]

# Get Sampsons network at T=4
G = ig.Graph.Read_GraphMLz(str(DATA/"t4.graphml.gz"))
B = Balance(G, beta=BETA, m=None, kmax=K)


/home/sztal/miniconda3/envs/msb/lib/python3.10/site-packages/igraph/io/files.py:189: RuntimeWarning: Could not add vertex ids, there is already an 'id' vertex attribute. at src/io/graphml.c:492
  return cls.Read_GraphML(tmpfile, index=index)


## Test trace power calculations

$$
\text{tr}A^k
$$

### Unsigned adjacency matrix

In [4]:
X = B.U

## CALCULATED VALUES
Y = np.real(np.exp(B.ltr_pow(X)))

## EXPECTED VALUE
E = np.array([ tr(mpow(X, k)) for k in B.K() ])

assert np.allclose(Y, E), "results do not match!"


### Signed adjacency matrix

In [5]:
X = B.S

## CALCULATED VALUES
Y = np.real(np.exp(B.ltr_pow(X)))

## EXPECTED VALUE
E = np.array([ tr(mpow(X, k)) for k in B.K() ])

assert np.allclose(Y, E), "results do not match!"


## Test trace exponential calculations

$$
\text{tr}e^{\beta{}A} = \left(
    \sum_{k=0}^\infty\frac{\beta^k}{k!}A^k
\right)
$$

### Unsigned adjacency matrix

In [6]:
X = B.U

## CALCULATED VALUES
Y = np.real(np.exp(B.ltr_texp(X, beta=BETA, K=B.K(0))))

## EXPECTED VALUES
E = np.array([ tr(texp(X*b)) for b in BETA ])

assert np.allclose(Y, E, rtol=1e-4), "results do not match!"


### Signed adjacency matrix

In [7]:
X = B.S

## CALCULATED VALUES
Y = np.real(np.exp(B.ltr_texp(X, beta=BETA, K=B.K(0))))

## EXPECTED VALUES
E = np.array([ tr(texp(X*b)) for b in BETA ])

assert np.allclose(Y, E), "results do not match!"


## Test diagonal powers calculations

$$
\text{diag}A^k
$$

### Unsigned adjacency matrix

In [8]:
X = B.U

## CALCULATED VALUES
Y = np.real(np.exp(B.ldiag_pow(X)))

## EXPECTED VALUES
E = np.column_stack([ mpow(X, k).diagonal() for k in B.K() ])

assert np.allclose(Y, E), "results do not match!"


### Signed adjacency matrix

In [9]:
X = B.S

## CALCULATED VALUES
Y = np.real(np.exp(B.ldiag_pow(X)))

## EXPECTED VALUES
E = np.column_stack([ mpow(X, k).diagonal() for k in B.K() ])

assert np.allclose(Y, E), "results do not match!"


## Test diagonal exponential calculations

$$
\text{diag}e^{\beta{}A} = \left(
    \sum_{k=0}^\infty\frac{\beta^k}{k!}A^k
\right)
$$

### Unsigned adjacency matrix

In [10]:
X = B.U

## CALCULATED VALUES
Y = np.real(np.exp(B.ldiag_texp(X, beta=BETA, K=B.K(0))))

## EXPECTED VALUES
E = np.column_stack([ texp(X*b).diagonal() for b in BETA ])


assert np.allclose(Y, E, rtol=1e-4), "results do not match!"


### Signed adjacency matrix

In [11]:
X = B.S

## CALCULATED VALUES
Y = np.real(np.exp(B.ldiag_texp(X, beta=BETA, K=B.K(0))))

## EXPECTED VALUES
E = np.column_stack([ texp(X*b).diagonal() for b in BETA ])


assert np.allclose(Y, E), "results do not match!"


## Test $PNP$ trace powers calculations

$$
\text{tr}\left(
    \sum_{l=1}^kP^{l-1}NP^{k-l}
\right)
$$

In [12]:
N = B.N
P = B.P

## CALCULATED VALUES
Y = np.real(np.exp(B.ltr_Vk()))

## EXPECTED VALUES
E = np.array([ tr(PNP_pow(N, P, k)) for k in B.K() ])

assert np.allclose(Y, E), "results do not match!"


## Test $PNP$ trace truncated exponential calculations

$$
\text{tr}\left(
    \sum_{k=0}^K\frac{\beta^k}{k!}\sum_{l=1}^k P^{l-1}NP^{k-l}
\right)
$$

In [13]:
N = B.N
P = B.P

## CALCULATED VALUES
Y = np.real(np.exp(B.ltr_V(beta=BETA)))

## EXPECTED VALUES
E = np.array([ tr(PNP_texp(N, P, *B.krange(), beta=b)) for b in BETA ])

assert np.allclose(Y, E), "results do not match!"


## Test $PNP$ diagonal powers calculations

$$
\text{diag}\left(
    \sum_{l=1}^k P^{l-1}NP^{k-l}
\right)
$$

In [14]:
N = B.N
P = B.P

## CALCULATED VALUES
Y = np.real(np.exp(B.ldiag_Vk()))

## EXPECTED VALUES
E = np.column_stack([ PNP_pow(N, P, k).diagonal() for k in B.K() ])

assert np.allclose(Y, E), "results do not match!"


## Test $PNP$ diagonal truncated exponential calculations

$$
\text{diag}\left(
    \sum_{k=0}^K\frac{\beta^k}{k!}\sum_{l=1}^k P^{l-1}NP^{k-l}
\right)
$$

In [15]:
N = B.N
P = B.P

## CALCULATED VALUES
Y = np.real(np.exp(B.ldiag_V()))

## EXPECTED VALUES
E = np.column_stack([ PNP_texp(N, P, *B.krange(), beta=b).diagonal() for b in BETA ])

assert np.allclose(Y, E), "results do not match!"


## Test matrix power calculations

$$
S^k = Q\Lambda^kQ^\top
$$

### Unsigned semiadjacency matrix

In [16]:
X = B.U

for k in B.K():
    Y = np.real(np.exp(B.lpow(X, k)))
    E = csr_array(mpow(X, k)).todense()
    assert np.allclose(Y, E), "results do not match!"


### Signed semiadjacency matrix

In [17]:
X = B.S

for k in B.K():
    Y = np.real(np.exp(B.lpow(X, k)))
    E = csr_array(mpow(X, k)).todense()
    assert np.allclose(Y, E), "results do not match!"


## Test matrix exponential calculations

$$
e^{\beta{}A} = \sum_{k=k_0}^{k_1}\frac{\beta^k}{k!}S^k
$$

### Unsigned semiadjacency matrix

In [18]:
X = B.U

for beta in BETA:
    Y = np.real(np.exp(B.ltexp(X, beta=beta)))
    E = csr_array(texp(X*beta, *B.krange())).todense()
    assert np.allclose(Y, E, rtol=1e-4), "results do not match!"


### Signed semiadjacency matrix

In [19]:
X = B.S

for beta in BETA:
    Y = np.real(np.exp(B.ltexp(X, beta=beta)))
    E = csr_array(texp(X*beta, *B.krange())).todense()
    assert np.allclose(Y, E, rtol=1e-2), "results do not match!"


## Test $PNP$ matrix power calclations

$$
\sum_{l=1}^kP^{l-1}NP^{k-l}
$$

In [20]:
N = B.N
P = B.P

for k in B.K():
    Y = np.real(np.exp(B.lnVk(k)))
    E = csr_array(PNP_pow(N, P, k)).todense()
    assert np.allclose(Y, E), "results do not match"


## Test $PNP$ matrix exponential calculations

$$
\sum_{k=k_0}^{k_1}\frac{\beta^k}{k!}\sum_{l=1}^kP^{l-1}NP^{k-l}
$$

In [21]:
N = B.N
P = B.P

for beta in BETA:
    Y = np.real(np.exp(B.lnV(beta=beta)))
    E = csr_array(PNP_texp(N, P, *B.krange(), beta=beta)).todense()
    assert np.allclose(Y, E), "results do not match"
